In [1]:
from PIL import Image
import numpy as np

In [2]:
image = Image.open('/content/sample_data/image.png')
image = np.array(image)
image

array([[[130, 118, 111, 255],
        [130, 118, 111, 255],
        [130, 118, 111, 255],
        ...,
        [ 56,  63,  78, 255],
        [ 56,  63,  78, 255],
        [ 55,  62,  78, 255]],

       [[127, 114, 107, 255],
        [127, 114, 107, 255],
        [127, 114, 107, 255],
        ...,
        [ 53,  60,  76, 255],
        [ 53,  60,  76, 255],
        [ 52,  59,  75, 255]],

       [[123, 111, 104, 255],
        [123, 111, 104, 255],
        [123, 111, 104, 255],
        ...,
        [ 51,  57,  73, 255],
        [ 50,  57,  73, 255],
        [ 50,  57,  73, 255]],

       ...,

       [[144, 130, 132, 255],
        [143, 129, 131, 255],
        [142, 128, 130, 255],
        ...,
        [ 43,  47,  64, 255],
        [ 43,  47,  64, 255],
        [ 43,  47,  64, 255]],

       [[144, 130, 132, 255],
        [143, 129, 131, 255],
        [142, 128, 131, 255],
        ...,
        [ 43,  47,  64, 255],
        [ 43,  47,  64, 255],
        [ 43,  47,  64, 255]],

       [[144

# Sharpen

In [3]:
def sharpen(image):
  # Get image details
  num_rows, num_cols, num_channels = image.shape

  # Create a 3x3 kernel
  kernel = [[0,-1,0],[-1,5,-1],[0,-1,0]]

  # Create an output image
  output = np.zeros_like(image)

  for channel in range(num_channels):
    for row in range(num_rows):
      for col in range(num_cols):

        # Apply the kernel to the current pixel with mirroring
        sum = 0

        for i in range(-1, 2):
          for j in range(-1, 2):
            # Since the kernel is equal to its flipped counterpart this works
            sum += image[mirror(row+i,num_rows)][mirror(col+j,num_cols)][channel] * kernel[i][j]

        output[row][col][channel] = sum

  return output

In [8]:
def mirror(number, bound):
  if number < 0:
    return abs(number)
  if number < bound:
    return number

  return bound - (number % bound)

In [5]:
result = sharpen(image)

In [7]:
output = Image.fromarray(result)
output.save('output.png')

Grayscale

In [9]:
def grayscale(image):
  # Get image details
  num_rows, num_cols, num_channels = image.shape

  # Create conversion kernels for each channel
  red_Kernel = [[-1,-1,1],[0,1,-1],[0,1,1]]
  green_Kernel = [[1,0,0],[1,-1,-1],[1,0,-1]]
  blue_Kernel = [[0,1,1],[0,1,0],[1,-1,1]]
  kernel_List = [red_Kernel, green_Kernel, blue_Kernel]

  # Create an output image
  output = np.empty([num_rows, num_cols])

  for channel in range(3):
    for row in range(num_rows):
      for col in range(num_cols):

        # Apply the kernel to the current pixel with mirroring
        sum = 0

        for i in range(-1, 2):
          for j in range(-1, 2):
            sum += int(image[mirror(row+i,num_rows)][mirror(col+j,num_cols)][channel] * kernel_List[channel][1-i][1-j])

          output[row][col] += sum

  return output

In [10]:
def scale (image):
  max = 0
  num_rows, num_cols = image.shape
  for row in range(num_rows):
    for col in range(num_cols):
      if image[row][col] > max:
        max = image[row][col]

  for row in range(num_rows):
    for col in range(num_cols):
      image[row][col] = image[row][col]/max
      image[row][col] = int(image[row][col]*255)

In [11]:
result = grayscale(image)

IndexError: index 2669 is out of bounds for axis 0 with size 2669

In [ ]:
result

array([[4.46000000e+02, 4.35000000e+02, 4.15000000e+02, ...,
        1.39900126e+23, 1.31635985e+23, 1.25733027e+23],
       [1.19830069e+23, 1.24552435e+23, 1.30455393e+23, ...,
        1.49935155e+23, 1.52296358e+23, 1.54657541e+23],
       [1.57018724e+23, 1.49935155e+23, 1.46393380e+23, ...,
        1.47573972e+23, 1.46983676e+23, 1.46983676e+23],
       ...,
       [4.69285266e+22, 4.60430829e+22, 4.81091182e+22, ...,
        3.11381088e+22, 2.95147953e+22, 3.05478130e+22],
       [2.62681684e+22, 6.46374009e+22, 6.10956260e+22, ...,
        3.64507711e+22, 3.48274576e+22, 3.39420139e+22],
       [3.36468660e+22, 3.24662744e+22, 2.99575172e+22, ...,
        4.22061601e+22, 4.22061601e+22, 3.92546810e+22]])

In [ ]:
scale(result)

In [ ]:
result

array([[0., 0., 0., ..., 7., 6., 6.],
       [5., 6., 6., ..., 7., 7., 7.],
       [8., 7., 7., ..., 7., 7., 7.],
       ...,
       [2., 2., 2., ..., 1., 1., 1.],
       [1., 3., 2., ..., 1., 1., 1.],
       [1., 1., 1., ..., 2., 2., 2.]])

In [ ]:
grayscale_output = Image.fromarray(np.uint8(result), 'L')
grayscale_output.save('grayscale_output.png')

Edge Detection

In [15]:
def edge (image):
  sobelx = [[-1,0,1],[-2,0,2],[-1,0,1]]
  sobely = [[1,2,1],[0,0,0],[-1,-2,-1]]

  num_rows, num_cols, num_channels = image.shape

  output = np.copy(image)

  for row in range(1, num_rows-1):
    for col in range (1, num_cols-1):
      for channel in range(num_channels):
        sum = 0

        for i in range(-1, 2):
          for j in range(-1, 2):
            sum += image[row+i][col+j][channel] * sobelx[1-i][1-j]
            sum += image[row+i][col+j][channel] * sobely[1-i][1-j]
            output[row][col][channel] = sum

  return output

In [16]:
result = edge(image)

In [17]:
output = Image.fromarray(result)
output.save('output.png')